In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://www.youtube.com/watch?v=NcaGFp76BTY

import shutil as sh
- shutil: 파일과 디렉터리를 복사, 이동, 삭제하는 등의 작업을 수행하는 라이브러리입니다.
- 예시: 파일 복사, 디렉터리 이동, 파일 삭제 등.
코드에서 파일 관리 작업을 보다 쉽게 처리하기 위해 사용될 수 있습니다.

from base64 import b64encode
- base64.b64encode: 데이터를 Base64로 인코딩하는 기능을 제공합니다. 주로 이진 데이터를 텍스트로 변환하는 데 사용됩니다.
- 예시: 이미지를 Base64 문자열로 인코딩하여 웹에서 전송하거나, HTML 내에 이미지를 인라인으로 포함시킬 때 사용됩니다.

In [2]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm import tqdm
import shutil as sh

from base64 import b64encode ## 이진 파일(예 비디오 파일)을 텍스트 형식으로 변환하여 html 문서나 소스 코드 내에 직접 포함시킬 수 있도록..


In [19]:
!pwd
!rm -rf yolov5

/content


In [3]:
%%time
!git clone https://github.com/ultralytics/yolov5
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt
!cp yolov5/requirements.txt ./

Cloning into 'yolov5'...
remote: Enumerating objects: 16965, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 16965 (delta 81), reused 98 (delta 49), pack-reused 16805 (from 1)
Receiving objects: 100% (16965/16965), 15.70 MiB | 18.48 MiB/s, done.
Resolving deltas: 100% (11610/11610), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
CPU times: user 153 ms, sys: 24.3 ms, total: 177 ms
Wall time: 15 s


In [4]:
!ls

drive  requirements.txt  sample_data  yolov5


In [5]:
%pwd

'/content'

Google Colab에서 유튜브 동영상을 다운 로드하기 위해 yt-dlp 라이브러리 이용

In [6]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.1 MB/s eta 0:00:00


In [7]:
import yt_dlp

## 유튜브 URL
url='https://www.youtube.com/watch?v=NcaGFp76BTY'

ydl_opts={'outtmpl': 'downloaded_video.mp4'}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
  ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=NcaGFp76BTY
[youtube] NcaGFp76BTY: Downloading webpage
[youtube] NcaGFp76BTY: Downloading ios player API JSON
[youtube] NcaGFp76BTY: Downloading web creator player API JSON
[youtube] NcaGFp76BTY: Downloading player d9418494
[youtube] NcaGFp76BTY: Downloading m3u8 information
[info] NcaGFp76BTY: Downloading 1 format(s): 136+251
[download] Destination: downloaded_video.mp4.f136.mp4
[download] 100% of    7.52MiB in 00:00:00 at 17.52MiB/s  
[download] Destination: downloaded_video.mp4.f251.webm
[download] 100% of   27.29KiB in 00:00:00 at 195.60KiB/s 
[Merger] Merging formats into "downloaded_video.mp4.mkv"
Deleting original file downloaded_video.mp4.f251.webm (pass -k to keep)
Deleting original file downloaded_video.mp4.f136.mp4 (pass -k to keep)


In [8]:
!ls

downloaded_video.mp4.mkv  drive  requirements.txt  sample_data	yolov5


In [9]:
###  ffmpeg은 멀티미디어 파일(동영상, 오디오)을 처리하는 데 사용, 다양한 포맷으로 변환하거나 오디오/비디오를 편집, 인코딩, 디코딩 가능
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [10]:
## -c는 사용할 코덱을 지정하는 옵션, 여기서는 copy는 코덱을 지정하지 않고, 기존 인코딩을 그대로 복사하라는 의미
!ffmpeg -i downloaded_video.mp4.mkv -c copy downloaded_video.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [12]:
!ls

downloaded_video.mp4  downloaded_video.mp4.mkv	drive  requirements.txt  sample_data  yolov5


In [14]:
!pwd

/content


--source downloaded_video.mp4:

감지하려는 입력 파일을 지정합니다.
여기서는 downloaded_video.mp4 파일을 소스로 사용하고 있습니다. 이 파일은 앞서 yt-dlp를 사용해 다운로드한 유튜브 동영상 파일로, 객체 감지를 수행하려는 대상입니다.
이 옵션에는 이미지 파일, 비디오 파일, 디렉토리, 웹캠 스트림, 혹은 URL을 입력할 수 있습니다.
--weights yolov5s.pt:

YOLOv5 모델의 가중치 파일을 지정합니다.
여기서 yolov5s.pt는 YOLOv5의 'small' 버전 가중치 파일을 의미합니다. YOLOv5에는 다양한 크기(버전)들이 있는데, yolov5s.pt는 성능은 중간 정도이지만 속도가 빠른 소형 모델입니다.
다른 가중치 파일로는 yolov5m.pt(medium), yolov5l.pt(large), yolov5x.pt(extra-large) 등이 있으며, 각각 크기와 정확도가 다릅니다.
--conf 0.25:

감지된 객체에 대한 신뢰도(confidence) 임계값을 설정합니다.
--conf 0.25는 탐지된 객체의 신뢰도 점수가 25% 이상일 때만 결과에 포함시키는 것을 의미합니다. 즉, 신뢰도가 0.25(25%) 미만인 객체는 무시됩니다.
이 값을 높이면 더 확실한 객체들만 감지하게 되고, 낮추면 더 많은 객체가 감지되지만 잘못된 감지가 늘어날 수 있습니다.

In [13]:
#### YOLOv5(Object Detection) 모델을 사용하여 downloaded_video.mp4 파일에서 객체를 감지
#### !python: Google Colab에서 셀 명령어로 실행
#### yolov5/detect.py: YOLOv5에서 제공하는 객체 탐지 스크립트 파일
#### -source downloaded_video.mp4: 감지하려는 입력 파일을 지정, 객체 감지를 수행 대상
#### weights yolov5s.pt: YOLOv5 모델의 가중치 파일을 지정
#### conf 0.25는 탐지된 객체의 신뢰도 점수가 25% 이상일 때만 결과에 포함시키는 것을 의미. 즉, 신뢰도가 0.25(25%) 미만인 객체는 무시
!python yolov5/detect.py --source downloaded_video.mp4 --weights yolov5s.pt --conf 0.25

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['yolov5s.pt'], source=downloaded_video.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-369-g907bef2f Python-3.10.12 torch-2.4.1+cu121 CPU

100% 14.1M/14.1M [00:00<00:00, 136MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 1

In [16]:
!ls

downloaded_video.mp4	  drive		    sample_data  yolov5s.pt
downloaded_video.mp4.mkv  requirements.txt  yolov5


In [15]:
from google.colab import files
files.download('/content/yolov5/runs/detect/exp/downloaded_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###### shutil을 이용한 파일 복사
https://zephyrus1111.tistory.com/199

In [18]:
import shutil
shutil.copy('/content/yolov5/runs/detect/exp/downloaded_video.mp4', '/content/drive/MyDrive/KITA_2024/M8_Vision응용/data/output.mp4')

'/content/drive/MyDrive/KITA_2024/M8_Vision응용/data/output.mp4'